In [50]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import requests

In [20]:
free_proxy = [
    '68.183.48.146:10006',
    '202.131.65.110:80',
    '64.225.8.132:10000',
    '41.77.188.131:80',
    '67.43.228.252:1499',
    '50.231.172.74:80',
    '50.174.7.158:80',
    '50.122.86.118:80',
    '50.222.245.42:80',
    '50.207.199.86:80',
    '50.175.212.66:80',
    '50.222.245.40:80',
    '133.18.234.13:80'
    ] 

In [21]:
def get_proxy(proxies):
    try:
        proxy = random.choice(proxies)
        print(proxy)
        return proxy
    except Exception as e:
        print(f"Proxy {proxy} failed due to error {e}")
    print("No working proxy has been found")
    return None

In [36]:
chrome_options = Options()
# Grabbing free proxy
#PROXY = get_proxy(free_proxy)
chrome_options.add_argument("--incognito")
#chrome_options.add_argument("--proxy-server=%s" % PROXY)
chrome_options.add_experimental_option("detach", True)
chrome_options.headless = True # For debugging. SHould be set to True when running in an actual test case
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)
# Printing the proxy of the driver

In [9]:
print(selenium.__version__)

4.17.2


* Bloomberg Market News URL format: https://www.bloomberg.com/news/articles/2024-02-03/creditor-battles-are-hobbling-risky-debt-credit-weekly?srnd=markets-vp

* Wall Street Journal Market News URL format: https://www.wsj.com/finance/investing/wall-street-gets-laser-eyes-in-bid-for-bitcoin-etf-bucks-3b890de3?mod=finance_lead_story

* Yahoo Finance News URL format: https://finance.yahoo.com/news/analysts-made-financial-statement-dws-060237470.html


In [39]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [46]:
ua = UserAgent()
print(ua.random)

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.36


In [61]:
# The below url_parser function is used to parse the url and get the news from the respective source

def url_parser(url):
    url = url.split('/')
    keyword = url[2].split('.')
    source = keyword[1]

    if source == 'bloomberg':
        url = get_blb_news(url)
        return url
    elif source == 'wsj':
        url = get_wsj_news(url)
        return url
    elif source == 'yahoo':
        url = get_yf_news(url)
        return url
    else:
        return "Source not supported"

* Bloomberg

In [51]:
# Bloomberg news -> Cookies acceptance is required to access the news

def accept_cookies():
    url = "https://sourcepointcmp.bloomberg.com/wrapper/v2/choice/consent-all?hasCsp=true&accountId=1425&env=prod&includeCustomVendorsRes=true&metadata=%7B%22gdpr%22%3A%7B%22applies%22%3Atrue%7D%7D&propertyId=31489&withSiteActions=true&ch=44031679743938401199d44444831847b85&scriptVersion=4.15.1&scriptType=unified"

    payload = {}
    headers = {
    'authority': 'sourcepointcmp.bloomberg.com',
    'accept': '*/*',
    'accept-language': 'en-GB,en;q=0.9',
    'content-type': 'application/json',
    'origin': 'https://www.bloomberg.com',
    'priority': 'u=1, i',
    'referer': 'https://www.bloomberg.com/news/articles/2024-02-03/creditor-battles-are-hobbling-risky-debt-credit-weekly?srnd=markets-vp',
    'sec-ch-ua': '"Chromium";v="121", "Not A(Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code == 200:
        print("Cookies accepted successfully.")
    else:
        print("Failed to accept cookies.")

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("detach", True)
chrome_options.headless = True # For debugging. SHould be set to True when running in an actual test case
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)


def get_blb_news(url):

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    driver.implicitly_wait(2)

    try:
        for web_page in range(2):
            prev_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(5)
                curr_height = driver.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break
                prev_height = curr_height

            try:
                accept_cookies()
                #button = WebDriverWait(driver, 10).until(
                    #EC.element_to_be_clickable((By.XPATH,"/html/body/div/div[2]/div[5]/button[1]"))
                    #)
                # button = driver.find_element(By.CLASS_NAME, 'link.caas-button.collapse-button')
                # driver.implicitly_wait(5)
                # ActionChains(driver).move_to_element(button).click(button).perform()
                #button.click()
                
            except Exception as e:
                print(f"Error: {str(e)}")

        news_url = []
        
        hrefs = [element.get_attribute("href") for element in driver.find_elements(By.XPATH, "//*[@href]")]

        for href in hrefs:
            if href.endswith('.html'):
                news_url.append(href)

        news = pd.DataFrame(columns=['url', 'h1', 'text'])

        for url in news_url[:1]:
            if url.endswith('.html'):
                news_url.append(url)

            h1 = driver.find_element(By.TAG_NAME, 'h1')
            h1 = h1.text 

            p_tags = driver.find_elements(By.TAG_NAME, 'p')

            text_content = ""
            for p in p_tags:
                text_content += p.text + " "
            text_content = text_content.strip()

            news = news._append({'url': url, 'h1': ''.join(h1), 'text': text_content}, ignore_index=True)
            return news
        
    finally:    
        driver.quit()

In [ ]:
def get_blb_news(url):

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    driver.implicitly_wait(2)

    try:
        for web_page in range(2):
            interval = 1 
            prev_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(interval)
                curr_height = driver.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break
                prev_height = curr_height

            try:
                driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME,"sp_choice_type_11"))))
                #button = driver.find_element(By.CLASS_NAME, 'link.caas-button.collapse-button')
                # driver.implicitly_wait(5)
                # ActionChains(driver).move_to_element(button).click(button).perform()
                # button_click()
                
            except Exception as e:
                print(f"Error: {str(e)}")

        news_url = []
        
        hrefs = [element.get_attribute("href") for element in driver.find_elements(By.XPATH, "//*[@href]")]

        for href in hrefs:
            news_url.append(href)

        news = pd.DataFrame(columns=['url', 'h1', 'text'])

        for url in news_url[:1]:
            if url.endswith('.html'):
                news_url.append(url)

            h1 = driver.find_element(By.TAG_NAME, 'h1')
            h1 = h1.text 

            p_tags = driver.find_elements(By.TAG_NAME, 'p')

            text_content = ""
            for p in p_tags:
                text_content += p.text + " "
            text_content = text_content.strip()

            news = news._append({'url': url, 'h1': ''.join(h1), 'text': text_content}, ignore_index=True)
            return news
        
    finally:    
        driver.quit()

In [ ]:
//*[@id="notice"]/div[5]/button[1]

<button title="Yes, I Accept" aria-label="Yes, I Accept" class="message-component message-button no-children focusable buttons-row sp_choice_type_11" style="opacity: 1; padding: 12px 30px; margin: 10px; border-width: 1px; border-color: rgb(0, 0, 0); border-radius: 0px; border-style: solid; font-size: 16px; font-weight: 700; color: rgb(255, 255, 255); font-family: arial, helvetica, sans-serif; width: auto; background: rgb(0, 0, 0);">Yes, I Accept</button>

/html/body/div/div[2]/div[5]/button[1]

In [52]:
blb_news = get_blb_news('https://www.bloomberg.com/news/articles/2024-02-06/goldman-morgan-stanley-bet-on-india-stocks-as-wall-street-shifts-from-china?srnd=premium-asia')

Failed to accept cookies.
Failed to accept cookies.


Exception ignored in: <function Service.__del__ at 0x000001F3BB3C6310>
Traceback (most recent call last):
  File "c:\Users\USER\OneDrive\Documents\YBIGTA\Project\practice-venv\lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\Users\USER\OneDrive\Documents\YBIGTA\Project\practice-venv\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\USER\OneDrive\Documents\YBIGTA\Project\practice-venv\lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\USER\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 214, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\USER\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "C:\Users\USER\AppData\Local

In [46]:
blb_news

In [12]:
#def get_proxy(proxies, c_options):
    #while True:  

        #proxy = random.choice(proxies)
        #print(proxy)
        #chrome_options.add_argument("--incognito")
        #chrome_options.add_argument("--proxy-server=%s" % proxy)
        #chrome_options.add_experimental_option("detach", True)
        #chrome_options.headless = False  # For debugging. Should be set to True when running in an actual test case
        #chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
        #driver = webdriver.Chrome(options=c_options)

        #try:
            #news_data = get_blb_news('https://www.bloomberg.com/news/articles/2024-02-06/goldman-morgan-stanley-bet-on-india-stocks-as-wall-street-shifts-from-china?srnd=premium-asia')
            #if news_data is not None:
                #return news_data, proxy
            #else:
                #print('Failed to retrieve news data')

        #except (TimeoutException, WebDriverException) as e:  # This error message is not getting caught for some reason...
            #print(f"ERROR MESSAGE: {e} and the proxy ip port {proxy} has been blocked")

        #finally:
            #driver.quit()

In [13]:
#test = get_proxy(free_proxy, chrome_options)

50.231.172.74:80


In [107]:
blb_news['text'][0]

'Investors are paying close attention to the contrasting trajectories of two of Asia’s greatest powers.  A momentous shift is under way in global markets as investors pull billions of dollars from China’s sputtering economy, two decades after betting on the country as the world’s biggest growth story. Much of that cash is now heading for India, with Wall Street giants like Goldman Sachs Group Inc. and Morgan Stanley endorsing the South Asian nation as the prime investment destination for the next decade.'

In [23]:
blb_news['text'][0]

'Listen 6:13 Chinese shares rallied after authorities intensified rescue efforts, defying broader weakness in Asia as hopes ebbed for a swift Federal Reserve pivot toward policy easing. The Hang Seng Index rose 2% while the mainland benchmark of CSI 300 was up 1.8%. The gains came after Beijing took more steps to stem a stock rout, such as widening trading curbs on investors including quantitative funds. A pledge by the nation’s sovereign wealth fund to further increase holdings of exchange-traded funds also lifted investor mood. In contrast, equities in Japan, Australia and South Korea fell, while US equity futures were steady after the S&P 500 dropped 0.3% and the Nasdaq 100 shed 0.2% Monday. “Right now the market is looking for clearer signals on the economic recovery,” Marcella Chow, global market strategist for JPMorgan Asset Management, said on Bloomberg Television regarding China. “Expectations remain quite low — markets and investors are still grappling with the weak economic r

* WSJ

In [343]:
# Wall Street Journal -> bypassing the bot detection 

#import undetected_chromedriver as uc

def get_wsj_news(url):
    #ua = UserAgent()
    #ua_random = ua.random

    #headers = {
        #'user-agent': ua_random,
        #'accept-language': 'en-US'
    #}

    #is_captcha = True
    #while is_captcha:
        #try:
            #driver = uc.Chrome
            #driver.get(url)
            #time.sleep(3)
            #soup = BeautifulSoup(driver.page_source, 'html.parser')
            #if 'captcha' in str(soup):
                #print('Bot detected')
                #time.sleep(3)  
            #else:
                #print('Bot bypassed')
                #is_captcha = False
                #return soup
        #except Exception as e:
            #print(f'Retrying due to an exception: {str(e)}')

* Yahoo Finance News

In [432]:
def get_yf_news(url):

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    driver.implicitly_wait(2)

    try:
        for web_page in range(2):
            interval = 1 
            prev_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(interval)
                curr_height = driver.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break
                prev_height = curr_height

            try:
                driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,"collapse-button"))))
                #button = driver.find_element(By.CLASS_NAME, 'link.caas-button.collapse-button')
                # driver.implicitly_wait(5)
                # ActionChains(driver).move_to_element(button).click(button).perform()
                # button_click()
                
            except Exception as e:
                print(f"Error: {str(e)}")

        news_url = []
        
        hrefs = [element.get_attribute("href") for element in driver.find_elements(By.XPATH, "//*[@href]")]

        for href in hrefs:
            if href.endswith('.html'):
                news_url.append(href)

        news = pd.DataFrame(columns=['url', 'h1', 'text'])

        for url in news_url[:1]:
            if url.endswith('.html'):
                news_url.append(url)

            h1 = driver.find_element(By.TAG_NAME, 'h1')
            h1 = h1.text 

            p_tags = driver.find_elements(By.TAG_NAME, 'p')

            text_content = ""
            for p in p_tags:
                text_content += p.text + " "
            text_content = text_content.strip()

            news = news._append({'url': url, 'h1': ''.join(h1), 'text': text_content}, ignore_index=True)
            return news
        
    finally:    
        driver.quit()

In [433]:
yf_news = get_yf_news("https://finance.yahoo.com/news/analysts-made-financial-statement-dws-060237470.html")

Error: Message: 



In [434]:
yf_news

,url,h1,text
0,https://finance.yahoo.com/news/analysts-made-f...,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ..."


In [419]:
def split_text(text):
    
    max_length = 750
    split_texts = []
    start = 0
    end = max_length

    while end < len(text):
        while text[end] not in ['.','!','?', ':', ';', '()']:
            end -= 1
        split_texts.append(text[start:end+1])
        start = end + 1
        end = start + max_length

    split_texts.append(text[start:])
    text_segments = pd.DataFrame(
        {
            'article': [text] * len(split_texts),
            'segment': split_texts
        }
    )

    return text_segments

In [420]:
def combine_texts(news):

    combined_data = {
        'h1': [], 
        'article': [], 
        'segment': []
    }

    for article, h1 in zip(news['text'], news['h1']):
        text_result = split_text(article)
        combined_data['h1'].extend([h1] * len(text_result['segment']))
        combined_data['article'].extend([text_result['article']] * len(text_result['segment']))
        combined_data['segment'].extend(text_result['segment'])

    combined_df = pd.DataFrame(combined_data)
    combined_df['article'] = combined_df['article'].iloc[1]
    return combined_df

In [421]:
combined_text = combine_texts(yf_news)
combined_text

,h1,article,segment
0,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ...","Last week, you might have seen that DWS Group ..."
1,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ...",Check out our latest analysis for DWS Group G...
2,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ...","84 price target, showing that the analysts don..."
3,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ...",It's pretty clear that there is an expectatio...
4,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ...",The most important thing to take away is that...
5,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ...",We have estimates - from multiple DWS Group G...
6,Analysts Have Made A Financial Statement On DW...,"Last week, you might have seen that DWS Group ...",It does not constitute a recommendation to bu...


In [ ]:
def prompt_for_summary(text):
    task = '''
    Summarise the given text in no more than 200 words. 
    The summary should be written in plain English and should filter out jargons for them to be displayed at the end in a numbered bulletpoint format.
    '''
    return task + text
